# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate
import numpy as np

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

Using default 16 lambda workers.

To maximize the degree of parallelism, add the following code to the beginning of the program:

"turicreate.config.set_runtime_config('TURI_DEFAULT_NUM_PYLAMBDA_WORKERS', 24)"

Note that increasing the degree of parallelism also increases the memory footprint.

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 2.283659     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 2.763683     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [6]:
model_all.coefficients.print_rows(num_rows=20)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.0559504957  |  None  |
|     bedrooms     |  None |        0.0         |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None |  8468.53108691009  |  None  |
|   sqft_living    |  None | 24.42072098244549  |  None  |
| sqft_living_sqrt |  None | 350.0605533860578  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 842.0680348976116  |  None 

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [7]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [8]:
l1_penalties = np.logspace(1,7, num=13)
rss_list = []
for l1_penalty in l1_penalties:
    model = turicreate.linear_regression.create(training, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=l1_penalty,
                                               verbose=False)
    predictions = model.predict(validation)
    rss = predictions - validation['price']
    rss = rss**2
    rss_list.append(rss.sum())

In [9]:
idx = np.argmin(rss_list)
l1_penalties[idx]

10.0

*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [10]:
rss_list

[625766285142460.4,
 625766285362394.0,
 625766286057885.2,
 625766288257224.9,
 625766295212186.0,
 625766317206080.6,
 625766386760658.0,
 625766606749278.6,
 625767302791635.0,
 625769507643885.9,
 625776517727024.5,
 625799062845466.6,
 625883719085425.2]

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [11]:
model_best = turicreate.linear_regression.create(training, target='price', features=all_features,\
                                                l2_penalty=0., l1_penalty=l1_penalties[idx],\
                                                validation_set=None, verbose=False)
model_best.coefficients.print_rows(num_rows=18)
model_best.coefficients['value'].nnz()

+------------------+-------+-----------------------+--------+
|       name       | index |         value         | stderr |
+------------------+-------+-----------------------+--------+
|   (intercept)    |  None |   18993.427212770588  |  None  |
|     bedrooms     |  None |   7936.967679031299   |  None  |
| bedrooms_square  |  None |   936.9933681933004   |  None  |
|    bathrooms     |  None |   25409.588934120664  |  None  |
|   sqft_living    |  None |   39.11513637970764   |  None  |
| sqft_living_sqrt |  None |   1124.6502128077168  |  None  |
|     sqft_lot     |  None | 0.0034836182229897833 |  None  |
|  sqft_lot_sqrt   |  None |   148.2583910114083   |  None  |
|      floors      |  None |   21204.335466950128  |  None  |
|  floors_square   |  None |   12915.524336072433  |  None  |
|    waterfront    |  None |   601905.5945452718   |  None  |
|       view       |  None |   93312.85731187186   |  None  |
|    condition     |  None |    6609.03571244722   |  None  |
|      g

18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [12]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [34]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [47]:
nnz = []
for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training, target='price', features=all_features,\
                                               l2_penalty=0., l1_penalty=l1_penalty,\
                                               validation_set=None, verbose=False)
    #nnz.append((l1_penalty, model.coefficients['value'].nnz()))
    nnz.append(model.coefficients['value'].nnz())

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [46]:
nnz = np.array(nnz)
#less_nonzeros = (filter(lambda x: x[1] < 7, nnz))
#more_nonzeros = (filter(lambda x: x[1] > 7, nnz))
#less_nonzeros = l1_penalty_values[np.where(nnz < 7)]
#more_nonzeros = l1_penalty_values[np.where(nnz > 7)]
l1_penalty_min = max([data[0] for data in less_nonzeros])
l1_penalty_max = min([data[0] for data in more_nonzeros])
print('Minimun l1 value: ', l1_penalty_min)
print('Maximum l1 value: ', l1_penalty_max)

Minimun l1 value:  10000000000.0
Maximum l1 value:  100000000.0


In [52]:
nnz = np.array(nnz)
less_nnz = l1_penalty_values[np.where(nnz < 7)]
more_nnz = l1_penalty_values[np.where(nnz > 7)]
l1_penalty_max, l1_penalty_min = np.min(less_nnz), np.max(more_nnz)
print('Minimun l1 value: ', l1_penalty_min)
print('Maximum l1 value: ', l1_penalty_max)

Minimun l1 value:  2976351441.6313133
Maximum l1 value:  3792690190.7322536


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [53]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [54]:
rss_list = []
nnz_list = []
l1_penalty_list = []
for l1_penalty in l1_penalty_values:
    model = turicreate.linear_regression.create(training, target='price', features=all_features,\
                                               l2_penalty=0., l1_penalty=l1_penalty,\
                                               validation_set=None, verbose=False)
    nnz = model.coefficients['value'].nnz()
    if nnz == max_nonzeros:
        predictions = model.predict(validation)
        errors = (predictions - validation['price'])**2
        rss_list.append(errors.sum())
        l1_penalty_list.append(l1_penalty)

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [55]:
idx = np.argmin(rss_list)
l1_penalty_list[idx]

3448968612.163437

In [56]:
model_best = turicreate.linear_regression.create(training, target='price', features=all_features,\
                                                l2_penalty=0., l1_penalty=l1_penalty_list[idx],\
                                                validation_set=None, verbose=False)
model_best.coefficients.print_rows(num_rows=18)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 222253.19254432796 |  None  |
|     bedrooms     |  None | 661.7227177822242  |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 15873.957259267983 |  None  |
|   sqft_living    |  None | 32.41022145125972  |  None  |
| sqft_living_sqrt |  None | 690.1147733133171  |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 2899.4202697498763 |  None 

In [97]:
np.min(rss_list)

1081867592324112.4

In [98]:
l1_penalty_list

[3792690190.7322536,
 4119390707.0095034,
 4446091223.286753,
 4772791739.564003,
 5099492255.841253,
 5426192772.118503,
 5752893288.395752,
 6079593804.673002,
 6406294320.950253,
 6732994837.227502,
 7059695353.504751,
 7386395869.7820015,
 7713096386.059252,
 8039796902.336501,
 8366497418.61375,
 8693197934.891,
 9019898451.168251,
 9346598967.445501,
 9673299483.72275,
 10000000000.0]